# Load HMI data into lvl1 and lvl2 datacubes

In [1]:
# Plotting packages
import matplotlib.pyplot as plt

import os
import natsort  # Sorting package. Used when I parse the folder for files. There may be simpler ways to do this.
import numpy as np

import astropy.units as u
import astropy.wcs
from astropy.coordinates import SkyCoord, SpectralCoord

import stokespy

# Set working directory. 
# This should be automated.
work_dir = os.getcwd()

data_dir = work_dir + '/Data/SDO/'

script_name = 'HMI_data_loading'

from ndcube import NDCube, GlobalCoords, ExtraCoords

In [4]:
def parse_folder(dir_path=None, inst=None, wave=None, ext=None, 
                 series=None, repo='JSOC', show=False):
    '''
    Search all the filenames in a folder containing SDO data and use the keywords
    to select a desired subset. For example setting inst="hmi" will obtain all the hmi filenames.
    inst: SDO instrument
    wave: wavelength (mostly for AIA data)
    ext: limit by file extension
    '''
    
    if dir_path is None:
        dir_path = os.getcwd()
    
    # Read and sort all the filenames in the folder.
    all_fnames = natsort.natsorted(os.listdir(dir_path)) 
    
    # Select a subset of files
    use_fnames = []
    
    for i, file_ in enumerate(all_fnames):
        if repo == 'JSOC':
            sfile_ = file_.lower().split(".")
        elif repo == 'VSO':
            sfile_ = file_.lower().split("_")
        
        if sfile_[0] == inst.lower() and sfile_[1] == series.lower(): 
            use_fnames.append(dir_path + file_)
        
        '''
        if ext is None and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is None and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is None:
            use_fnames.append(dir_path + file_)
        elif ext is not None and file_.endswith(ext) and inst is not None and sfile_[0] == inst.lower() and wave is not None and sfile_[2] == wave + 'a':
            use_fnames.append(dir_path + file_)
        '''
    '''
    for i, file_ in enumerate(all_fnames):
        # Select by extension.
        if ext is not None and file_.endswith(ext):        
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
        else:
            sfile_ = file_.split("_")
            # Select by instrument.
            if inst is not None and sfile_[0] == inst.lower(): 
                if wave is not None and inst.lower() == 'aia' and sfile_[2] == wave + 'a':   
                    use_fnames.append(dir_path + file_)
    '''
    
    if show:
        for i, file_ in enumerate(use_fnames):
            print(i, file_)
    
    return use_fnames

# Test the functiom.
'''
# Parse the data folder and select subset of files.
inst = 'aia'
wave = '171'

use_fnames = parse_folder(dir_path = img_dir, ext='fits', inst='aia', wave='171')
print(len(use_fnames))
for i in use_fnames:
    print(i)
'''

"\n# Parse the data folder and select subset of files.\ninst = 'aia'\nwave = '171'\n\nuse_fnames = parse_folder(dir_path = img_dir, ext='fits', inst='aia', wave='171')\nprint(len(use_fnames))\nfor i in use_fnames:\n    print(i)\n"

# Create StokesCube from HMI images

In [5]:
# Obtain a list of available HMI images 
# This needs to be improved to accept user imput for the date
# and type of series wanted. 
# Stokes series is labeled with: S_720s

all_fnames= parse_folder(dir_path=data_dir, inst='hmi', series='S_720s', ext='fits', show=True)

0 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I0.fits
1 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I1.fits
2 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I2.fits
3 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I3.fits
4 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I4.fits
5 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.I5.fits
6 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.Q0.fits
7 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.Q1.fits
8 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.s_720s.20160729_000000_TAI.3.Q2.fits
9 /home/ga

## Create data array

In [6]:
level1_data = []
wcs_obj = []
header_info = []

for i, fname in enumerate(all_fnames):
    with astropy.io.fits.open(fname) as hdulist:
        level1_data.append(hdulist[1].data)
        wcs_obj.append(astropy.wcs.WCS(hdulist[1].header))
        header_info.append(hdulist[1].header)
    
level1_data = np.asarray(level1_data)
level1_data = level1_data.reshape(4,6,level1_data.shape[1], level1_data.shape[2])

print(f'Created data cube with dimensions: {level1_data.shape}')

Created data cube with dimensions: (4, 6, 4096, 4096)


## Create WCS

In [7]:
# Expand the coordinate axis to include wavelength and stokes dimensions.

l0 = 6173.345 * 1.e-10  # m Central wavelength for FeI line
dl = 0.0688   * 1.e-10  # m 

# Generate WCS for data cube using same WCS celestial information from AIA map.
wcs_header = wcs_obj[0].to_header()

wcs_header["WCSAXES"] = 4

# Add wavelength axis.
wcs_header["CRPIX3"] = 3.5
wcs_header["CDELT3"] = dl
wcs_header["CUNIT3"] = 'm'
wcs_header["CTYPE3"] = "WAVE"
wcs_header["CRVAL3"] = l0

# Add Stokes axis.
wcs_header["CRPIX4"] = 0
wcs_header["CDELT4"] = 1
wcs_header["CUNIT4"] = ''
wcs_header["CTYPE4"] = "STOKES"
wcs_header["CRVAL4"] = 0

level1_wcs = astropy.wcs.WCS(wcs_header)

In [8]:
level1_wcs

WCS Keywords

Number of WCS axes: 4
CTYPE : 'HPLN-TAN'  'HPLT-TAN'  'WAVE'  'STOKES'  
CRVAL : 0.0  0.0  6.173345e-07  0.0  
CRPIX : 2039.1423339844  2051.4689941406  3.5  0.0  
PC1_1 PC1_2 PC1_3 PC1_4  : -0.99999997203655  0.00023648870259451  0.0  0.0  
PC2_1 PC2_2 PC2_3 PC2_4  : -0.00023648870259451  -0.99999997203655  0.0  0.0  
PC3_1 PC3_2 PC3_3 PC3_4  : 0.0  0.0  1.0  0.0  
PC4_1 PC4_2 PC4_3 PC4_4  : 0.0  0.0  0.0  1.0  
CDELT : 0.00014009369744195  0.00014009369744195  6.88e-12  1.0  
NAXIS : 0  0

In [9]:
level1_wcs.world_axis_units

['deg', 'deg', 'm', '']

## Create the StokesCube object

In [10]:
level1_cube = stokespy.StokesCube(level1_data, level1_wcs)

In [11]:
# Spectral axis checks out.
print('Input wavelength ') 
print((l0 + dl * (np.arange(0,6) - 3.5 + 1)))
print('StokesCube wavelength')
print(level1_cube.spectral_axis)

Input wavelength 
[6.1731730e-07 6.1732418e-07 6.1733106e-07 6.1733794e-07 6.1734482e-07
 6.1735170e-07]
StokesCube wavelength
[6.1731730e-07 6.1732418e-07 6.1733106e-07 6.1733794e-07 6.1734482e-07
 6.1735170e-07] m


# Create MagVectorCube from HMI inversions

In [12]:
# Obtain a list of available HMI images 
# This needs to be improved to accept user imput for the date
# and type of series wanted. 
# Stokes series is labeled with: S_720s

all_fnames = parse_folder(dir_path=data_dir, inst='hmi', series='me_720s_fd10', ext='fits', show=True)

0 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.alpha_err.fits
1 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.alpha_mag.fits
2 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.azimuth.fits
3 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.azimuth_alpha_err.fits
4 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.azimuth_err.fits
5 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.chisq.fits
6 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.confid_map.fits
7 /home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.conv_flag.fits
8 /home/gabriel/Desktop/Sci

## Create data array

In [13]:
mag_params = ['field', 'inclination', 'azimuth']

level2_data = []
wcs_obj = []
header_info = []

# Load the data in the order determined y
use_fnames = []
for mag_param in mag_params:
    for i, fname in enumerate(all_fnames):
        data_id = fname.split('.')[-2]
        if data_id == mag_param:
            use_fnames.append(fname)
            with astropy.io.fits.open(fname) as hdulist:
                level2_data.append(hdulist[1].data)
                wcs_obj.append(astropy.wcs.WCS(hdulist[1].header))
                header_info.append(hdulist[1].header)
            
level2_data = np.asarray(level2_data)
            
print(f'Created data cube with dimensions: {level2_data.shape}')
print('Filenames used: ')
for fname in use_fnames:
    print(fname)

Created data cube with dimensions: (3, 4096, 4096)
Filenames used: 
/home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.field.fits
/home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.inclination.fits
/home/gabriel/Desktop/Science/StokesPY/stokespy/notebooks/Data/SDO/hmi.me_720s_fd10.20160729_000000_TAI.azimuth.fits


## Create WCS

In [14]:
# Expand the wcs coordinates to include the magnetic field parameters.

# Generate WCS for data cube using same WCS celestial information from AIA map.
wcs_header = wcs_obj[0].to_header()

wcs_header["WCSAXES"] = 3

# Add Stokes axis.
wcs_header["CRPIX3"] = 0
wcs_header["CDELT3"] = 1
wcs_header["CUNIT3"] = ''
wcs_header["CTYPE3"] = "Parameter"
wcs_header["CRVAL3"] = 0

level2_wcs = astropy.wcs.WCS(wcs_header)

## Create the MagVectorCube object

In [15]:
level2_cube = stokespy.MagVectorCube(level2_data, level2_wcs)

In [17]:
# I use jupyter lab which uses this widget command for interactive plotting
#%matplotlib notebook
%matplotlib widget  

mag_map = level2_cube.B_map
mag_map.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<WCSAxesSubplot:>